In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def convertOutput(file):
    """The Output of gquad (the r package) is very weird, so this function converts that output to a semi readable format. More work needs to be done"""
    df = pd.read_csv(file)
    arr = df.to_numpy()
    shape = arr.shape
    arr = arr.reshape((-1, int(shape[0]/3)))
    return pd.DataFrame(arr).swapaxes('columns', 'rows')

In [ ]:
def avg_non_b_dna_ipd_one_file(nbdna_filename, ipd_filename, likeliness, length=300):
    ipd_df = pd.read_csv(ipd_filename)
    ipd_df = ipd_df[ipd_df['strand'] == 0]
    nbdna_df = pd.read_csv(nbdna_filename)
    nbdna_df = nbdna_df[nbdna_df['likeliness'] == likeliness]
    master_arr = 'None'
    for seq_pos, seq_len in zip(nbdna_df['sequence_position'], nbdna_df['sequence_length']):
        if master_arr is 'None': # '==' works too
            master_arr = np.array([ipd_df[ipd_df['tpl'] >= (seq_pos - ((length - seq_len) / 2))].head(length)['ipdRatio'].values])
        else:
            master_arr = np.append(
                master_arr,
                np.array([ipd_df[ipd_df['tpl'] >= (seq_pos - ((length - seq_len) / 2))].head(length)['ipdRatio'].values]),
                axis=0
            )
    
    # Plot some of them
    for c, n in enumerate(master_arr):
        plt.plot(n)
        plt.show()
        if c > 10:
            break
    return master_arr.mean(axis=0)

def avg_non_b_dna_ipd(ipd_dir, nbdna_dir, likeliness, length=300):
    master_arr = 'None'
    for n in os.listdir(nbdna_dir):
        # Getting file names
        if '.csv' in n:
            nbdna_filename = os.path.join(nbdna_dir, n)
        else:
            continue
        ipd_filename = os.path.join(ipd_dir, '_'.join(os.path.basename(nbdna_filename).split('.')[0].split('_')[1:]) + '.bam.ipd')

        # Debug Purposes. Remove
        if '2' in n:
            continue
        if '_1' in n:
            continue


        cont = False
        # Making sure its not empty file
        with open(nbdna_filename, 'r') as f:
            counter = 0
            for n in f.readlines():
                if '-' in n:
                    cont = True
                    break
                if counter >= 3:
                    cont = False
                    break
        
        if cont:
            continue

        # Getting IPDs
        nbdna_df = pd.read_csv(nbdna_filename)
        nbdna_df = nbdna_df[nbdna_df['likeliness'] == likeliness]
        ipd_df = pd.read_csv(ipd_filename)
        ipd_df = ipd_df[ipd_df['strand'] == 0]

        for seq_pos, seq_len in zip(nbdna_df['sequence_position'], nbdna_df['sequence_length']):
            if master_arr is 'None': # '==' works too\
                values = ipd_df[ipd_df['tpl'] >= (seq_pos - ((length - seq_len) / 2))].head(length)['ipdRatio'].values
                if values.__len__() == length:
                    master_arr = np.array([values])
            else:
                values = ipd_df[ipd_df['tpl'] >= (seq_pos - ((length - seq_len) / 2))].head(length)['ipdRatio'].values
                if values.__len__() == length:
                    master_arr = np.append(
                        master_arr,
                        np.array([values]),
                        axis=0
                    )
    return master_arr.mean(axis=0)

## IPD Values from Z-DNA

In [ ]:
ipds = avg_non_b_dna_ipd('ipds\ipd', 'R_gquad_results/zdna', '**', length=300)

In [ ]:
plt.plot(ipds)
plt.show()
np.savetxt("zdna-avg-ipds-300.csv", ipds, delimiter=",")

In [ ]:
ipds = avg_non_b_dna_ipd('ipds\ipd', 'R_gquad_results\zdna', '**', length=100)